<h1> Generate Network Models

In [120]:
import networkx as nx
import random
import os
import numpy as np

<h2> Tutorial

<h2> Workspace

<h1>Functions

<h2> 1. Fractal Models

<h3> 1.1 Song-Havlin-Makse (SHM) Model

A SHM network relies on two predetermined parameters, $m$ and $p$, and is generated as follows

1. Begin with two nodes connected by a single edge. 

2. At stage $n$, $m$ offspring are connected to each endpoint of every node. 

3. Also at stage $n$, each edge in the $(n-1)$-th generation is removed randomly with independent probability $p$ and replaced with an edge between two of the new offspring at the endpoints of this edge. 

In [121]:
def generate_SHM_model_for_all_p(m, N, prob_N=11, example_N=1, save=False):
    
    probabilities = np.linspace(0, 1, prob_N)
    
    graphs = []
    
    for p in probabilities:
        p_graphs = []
        for i in range(example_N):
            G = generate_SHM_model(m, p, N, save=save)
            p_graphs.append(G)
        graphs.append(p_graphs)
        
    return probabilities, graphs

In [118]:
def retrieve_SHM_model(m, p, N, example=1):
    """
    Given the parameters m, p, N and the example number, retrieve the file containing the SHM network. 
    
    Args:
        m (int): The number of offspring added at each stage, as defined by the SHM model [2].
        p (float): The probability of rewiring an edge, as defined by the SHM model [2]. 
        N (int): The number of iterations to perform of the SHM generative process [2].
        example (int): In the case of multiple graphs with the same parameters, specifies the example wanted. 
        
    Returns:
        G (networkx.Graph): The SHM Model with the above specified parameters. 
    """
    
    # Find the filepath to the model with these parameters. 
    filename = "SHM-model-" + str(m) + "-" + str(p) + "-generation" + str(N) + "-example" + str(example) + ".gml"
    filepath = "network-files/models/SHM-model/"

    # Read the network.
    G = nx.read_gml(filepath+filename)
    
    # Return the networkx graph.
    return G

In [241]:
def generate_SHM_model(m, p, N, save=False):
    G = nx.path_graph(2)
    
    for i in range(N-1):
        G = SHM_iteration(G, m, p)
        
    if save==True:
        # Save the file in the format uvflower-generationN.gml
        count = 1
        saved = False
        
        while saved == False:
            filename = "SHM-model-" + str(m) + "-" + str(p) + "-generation" + str(N) + "-example" + str(count) + ".gml"
            filepath = "network-files/models/SHM-model/" + filename
            if not os.path.isfile(filepath):
                nx.write_gml(G, filepath)
                saved=True
            else:
                count += 1
    
    return G

In [194]:
def SHM_iteration(G, m, p):
    
    edges = list(G.edges())
    
    for edge in edges:
        G, source_offspring = add_m_offspring(G, edge[0], m)
        G, target_offspring = add_m_offspring(G, edge[1], m)
        
        if random.random() <= p:
            rewire_offspring(G, edge, source_offspring, target_offspring)
        
    return G

In [192]:
def add_m_offspring(G, node, m):
    n = len(G.nodes())
    
    new_nodes = [n+i for i in range(m)]
    new_edges = zip(new_nodes, [node]*m)
    
    G.add_nodes_from(new_nodes)
    G.add_edges_from(new_edges)
    
    return G, new_nodes
    

In [200]:
def rewire_offspring(G, edge, source_offspring, target_offspring):
    G.remove_edge(edge[0], edge[1])
    
    new_source = random.choice(source_offspring)
    new_target = random.choice(target_offspring)
    
    G.add_edge(new_source, new_target)
    
    return G

<h3> 1.2 $(u, v)$-Flowers

A $(u, v)$-flower [1], with $1<u<v$, is generated as follows:

1. Begin with a cycle graph of length $w= u+v$.

2. In the $n$-th generation, replace each edge in the graph from the $(n-1)$-th generation with two parallel paths, one of length $u$ and the other of length $v$.

This network is self similar because the $n$-th generation contains $w$ copies of the $(n-1)$-th generation.

The model is also deterministic.

In [5]:
def generate_uv_flower(u, v, N):
    """
    Generates an n-th generation (u,v)-flower.
    
    Args:
        u (int): Value of u, i.e. path length of one of the parallel paths. 
        v (int): Value of v, i.e. path length of one of the parallel paths. 
        N (int): Number of generations.
        
    Returns:
        filepath (str): File path to .gml file containing generated network.
    """
    # Initialise a cycle graph of length w = u + v
    G = nx.cycle_graph(u+v)
    
    # For each of the n generations, perform one iteration of the generative process. 
    for i in range(N-1):
        G = uv_iteration(G, u, v)
        
    # Save the file in the format uvflower-generationN.gml
    filename = str(u) + str(v) + "flower-generation" + str(N) + ".gml"
    filepath = "network-files/models/uv-flowers/" + filename
    nx.write_gml(G, filepath)
    
    return filepath

In [7]:
def uv_iteration(G, u, v):
    """
    Performs one iteration in the (u,v)-flower generation process. 
    
    Args:
        G (networkx.Graph): The (u,v)-flower network in its current (t-1)-th generation. 
        u (int): Value of u, i.e. path length of one of the parallel paths. 
        v (int): Value of v, i.e. path length of one of the parallel paths. 
        
    Returns:
        G (networkx.Graph): The (u,v)-flower network in the t-th generation. 
        
    """
    # Find a list of all the nodes and edges in the network at the (t-1)-th generation.
    nodes = list(G.nodes())
    edges = list(G.edges())
    
    # Remove all the existing edges. 
    G.remove_edges_from(G.edges())

    # Iterate through each of the edges from the network in the (t-1)-th generation.
    for edge in edges:
        # n is used to store the smallest integer which isn't yet a node label. 
        # The nodes are labelled 0, ..., n-1, so this is n.
        n = len(G.nodes())
        
        # Replace the edge with a path of length u.
        # First find a path graph using these vertices.
        Hu, n = add_new_path(u, n, edge)
        # Then merge this path graph with the existing network.
        G = nx.compose(G, Hu)
        
        # Replace the edge with a path of length v.
        # First find a path graph using these vertices.
        Hv, n = add_new_path(v, n, edge)
        # Then merge this path graph with the existing network.
        G = nx.compose(G, Hv)

    # Return the graph after all iterations. 
    return G

In [15]:
def add_new_path(l, n, edge):
    """
    Adds a new parallel path to the network, a step in the (u, v)-flower generation process. 
    
    Args:
        l (int): The length of the path to be added to the network.
        n (int): A counter which stores the next unused integer to label nodes. 
        edge (tuple): The edge from the network being replaced with parallel paths. 
        
    Returns:
        Hl (networkx.Graph): A path graph of length l with vertices labelled correctly. 
        n (int): A counter which stores the next unused integer to label nodes. 
    """
    # Generate a path graph with l edges (and l+1 vertices). 
    Hl = nx.path_graph(l+1)
    
    # Create an empty dictionary to be used to relabel the nodes in the path. 
    l_rlbl = {key:None for key in list(Hl.nodes)}
    
    # The nodes in the path graph are labelled from 0 to l. 
    # Thus, the node 0 in this path corresponds to the source node of the original edge,
    #   and the node l in this path corresponds to the target node of the original edge.
    
    # Iterate through all the nodes in the path.
    for node in list(Hl.nodes()):
        # If the node is 0 in the path, relabel it as the source of the original edge.
        if node == 0:
            l_rlbl[node] = edge[0]
        # If the node is l in the path, relabel it as the source of the original edge.
        elif node == l:
            l_rlbl[node] = edge[1]
        # For all other nodes, relabel it as the next unused integer. 
        else:
            l_rlbl[node] = n
            # Increment the counter n, so that n is now the next unused integer. 
            n += 1
            
    # Relabel the nodes according to the scheme described above. 
    Hl = nx.relabel_nodes(Hl, l_rlbl)
    
    # Return the path graph to be merged with the (u, v)-flower graph, and the counter for the next unused integer for node labels. 
    return Hl, n

<h2> 2. Non-Fractal Models

<h2> 2.1 $(u, v)$-Flowers

It was shown in [1] that $(u, v)$-flowers with $u=1$ are non-fractal. 

In [101]:
def generate_non_fractal_uv_flower(v, N):
    """
    Generates a non-fractal (u,v)-flower with u=1.
    
    Args:
        v (int): Value of v, i.e. path length of the parallel paths. 
        N (int): Number of generations.
        
    Returns:
        filepath (str): File path to .gml file containing generated network.
    """
    # Returns the (u,v)-flower found by the fractal generator, but with u hardcoded as 1. 
    return generate_uv_flower(1, v, N)

<h1> References

[1] H. D. Rozenfeld, L. K. Gallos, C. Song, and H. A. Makse, “Fractal and transfractal scale-free networks,” in
Encyclopedia of Complexity and Systems Science. Springer New York, 2009, pp. 3924–3943